In [1]:
import musicbox.theory as th
import musicbox.box as box
import random
import mido

In [2]:
roman = ['I', 'II', 'III', 'IV', 'V', 'VI', 'VII']
tonleiter = list(th.Tone.all())
progression = [['I', 'V', 'IV', 'I'],
              ['I', 'IV', 'V', 'IV'],
              ['I', 'I', 'IV', 'V'],
              ['I', 'IV', 'I', 'V'],
              ['I', 'IV', 'V', 'V'],
              ['I', 'VI', 'II', 'V'],
              ['I', 'VI', 'IV', 'V'],
              ['I', 'IV', 'V', 'I'],
              ['I', 'V', 'I'],
              ['VI', 'II', 'V', 'I']]

def setting():
    out = dict()
    out['root'] = random.choice(list(th.Tone.all()))
    scale_types = ['major']  # TODO select all scales
    out['scale_type'] = random.choice(scale_types)
    out['scale'] = th.Scale(out['root'], out['scale_type'])
    out['rhythm'] = [random.choice([True, False]) for _ in range(8)]
    out['progression'] = random.choice(progression)
    return out

def select(tones, notes, previous=None):
    actual_notes = []
    for note in notes:
        for tone in tones:
            if note.tone == tone:
                actual_notes.append(note)
    return random.choice(actual_notes)

In [3]:
asdf = setting()
print(asdf['root'])
print(asdf['scale_type'])
print(repr(asdf['scale']))
print(asdf['rhythm'])
print(asdf['progression'])


A
major
Scale("A", "major")
[True, True, False, False, True, True, False, True]
['I', 'IV', 'V', 'I']


In [4]:
s = asdf['scale']
notes = []
for p in asdf['progression']:
    for r in asdf['rhythm']:
        notes.append(select(s.degree(p).tones, box.GI30.notes))

In [18]:
mid = mido.MidiFile()
track = mido.MidiTrack()
mid.tracks.append(track)
for name in mido.get_output_names():
    if 'FLUID' in name:
        port = mido.open_output(name)

In [19]:
track.append(mido.Message('program_change', program=10, time=0))
# TODO set tempo bpm2tempo

In [20]:
for i, note in enumerate(notes):
    track.append(mido.Message('note_on', note=note.midi, velocity=64, time=100))
    track.append(mido.Message('note_off', note=note.midi, velocity=127, time=100))

In [21]:
for msg in mid.play():
    port.send(msg)

In [22]:
port.close()

In [ ]:
# mid.save('test.mid')